# Complare the analytical and numerical solution of the Helmholtz problem with Dirichlet boundary condition

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
%matplotlib inline
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [3]:
d = [1,0,0] # wave's travel direction
k = 0.4 # wavenumber
radius = 2

big_sphere = bempp.api.shapes.sphere(r = radius+0.2, origin = [0,0,0], h = h)
vert_big_sphere = big_sphere.vertices

In [16]:
error_1 = []
error_2 = []
for num_of_truncate_term in [5,10,15,20,25,30]:
    for h in [0.2,0.1,0.05]:
        dom_sphere = bempp.api.shapes.sphere(r = radius, origin = [0,0,0], h = h)
        space_sphere = bempp.api.function_space(dom_sphere, 'P', 1)
        unit_sphere = bempp.api.shapes.sphere(r = 1, origin = [0,0,0], h = h)
        vert_unit_sphere = unit_sphere.vertices
        space_unit_sphere = bempp.api.function_space(unit_sphere,'P',1)
        
        eta = k
        identity = bempp.api.operators.boundary.sparse.identity(space_sphere, space_sphere, space_sphere)
        slp = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere, space_sphere, space_sphere,k)
        adlp = bempp.api.operators.boundary.helmholtz.adjoint_double_layer(space_sphere, space_sphere, space_sphere,k)
        slp_far = bempp.api.operators.far_field.helmholtz.single_layer(space_sphere, vert_unit_sphere, k)

        lhs = 0.5 * identity + adlp - 1j *eta* slp
        
        @bempp.api.complex_callable
        def combined_data(x, n, domain_index, result):
            result[0] = 1j * k * np.exp(1j * k * x[0]) * n[0] - 1j * eta * np.exp(1j * k * x[0])

        grid_fun = bempp.api.GridFunction(space_sphere, fun=combined_data)
        neumann_fun, info = bempp.api.linalg.gmres(lhs, grid_fun, tol=1E-5)

        res_far = -slp_far.evaluate(neumann_fun)
        err1 = np.linalg.norm(far_field(vert_unit_sphere[:,0]) - res_far[0,0])/np.linalg.norm(far_field(vert_unit_sphere[:,0]))
        error_1.append(err1)
        
        eta2 = k
        dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_sphere,space_sphere,space_sphere,k)
        dlp_far =  bempp.api.operators.far_field.helmholtz.double_layer(space_sphere, vert_unit_sphere,k)

        lhs2 = 0.5 * identity + dlp - 1j* eta2 *slp
        
        @bempp.api.complex_callable
        def data(x, n, domain_index, result):
            result[0] = - np.exp(1j * k * x[0])

        grid_fun2 = bempp.api.GridFunction(space_sphere, fun=data)
        fun2, info2 = bempp.api.linalg.gmres(lhs2, grid_fun2, tol=1E-5)

        res_far2 = dlp_far.evaluate(fun2) - 1j * eta2 * slp_far.evaluate(fun2)
        err2 = np.linalg.norm(far_field(vert_unit_sphere[:,0]) - res_far2[0,0])/np.linalg.norm(far_field(vert_unit_sphere[:,0]))
        error_2.append(err2)
        print(num_of_truncate_term,h,err1,err2)

5 0.2 0.00526185562185891 0.005262169940553443
5 0.1 0.0015148517149452128 0.0015147401883953775
5 0.05 0.0004119371114077413 0.00041197086692920394
10 0.2 0.0052618556533394955 0.005262169972036515
10 0.1 0.00151485174656705 0.001514740220023193
10 0.05 0.00041193714310318227 0.00041197089862817795
15 0.2 0.0052618556533394955 0.005262169972036515
15 0.1 0.00151485174656705 0.001514740220023193
15 0.05 0.00041193714310318227 0.00041197089862817795
20 0.2 0.0052618556533394955 0.005262169972036515
20 0.1 0.00151485174656705 0.001514740220023193
20 0.05 0.00041193714310318227 0.00041197089862817795
25 0.2 0.0052618556533394955 0.005262169972036515
25 0.1 0.00151485174656705 0.001514740220023193
25 0.05 0.00041193714310318227 0.00041197089862817795
30 0.2 0.0052618556533394955 0.005262169972036515
30 0.1 0.00151485174656705 0.001514740220023193
30 0.05 0.00041193714310318227 0.00041197089862817795


In [17]:
error_1

[0.00526185562185891,
 0.0015148517149452128,
 0.0004119371114077413,
 0.0052618556533394955,
 0.00151485174656705,
 0.00041193714310318227,
 0.0052618556533394955,
 0.00151485174656705,
 0.00041193714310318227,
 0.0052618556533394955,
 0.00151485174656705,
 0.00041193714310318227,
 0.0052618556533394955,
 0.00151485174656705,
 0.00041193714310318227,
 0.0052618556533394955,
 0.00151485174656705,
 0.00041193714310318227]

### Analytical solution of the scatterd field / far field pattern scattered by the incident plane wave $e^{ikx\cdot d}$ on the unit sphere 

In [12]:
def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def spherical_hankel_function(q,x):
    """Spherical hankel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.hankel1(q+0.5, k*r)

def Legendre_poly(q, x):  
    if(q == 0): 
        return 1 # P0 = 1 
    elif(q == 1): 
        return x # P1 = x 
    else: 
        return ((2*q - 1) * x * Legendre_poly(q-1, x) - (q-1)*Legendre_poly(q - 2, x))/q 

#### Compare the scatterd field

In [4]:
def scattered_field(x):
    field = 0
    cosine_angle = x[0]/np.linalg.norm(x)
    for i in range(num_of_truncate_term):
        a = -((1j)**i) * (2*i+1) * spherical_bessel_function(i,[radius,0,0]) * spherical_hankel_function(i,x) * Legendre_poly(i,cosine_angle) / spherical_hankel_function(i,[radius,0,0])
        field += a
    return field

scattered_field(vert_big_sphere[:,0])

(0.4548516231484017+0.8968920878502571j)

#### Compare the far field

In [14]:
def far_field(x):
    field = 0
    cosine_angle = x[0]/np.linalg.norm(x)
    for i in range(num_of_truncate_term):
        a = (1j/k) * (2*i+1) * spherical_bessel_function(i,[radius,0,0]) * Legendre_poly(i,cosine_angle) / spherical_hankel_function(i,[radius,0,0])
        field += a
    return field

far_field(vert_unit_sphere[:,0])

(-0.9341853203569978+0.3118077511365322j)

In [20]:
@bempp.api.complex_callable
def far_fun(x,n,domain_index,result):
    with objmode():
        result[0] = far_field(x)
far_grid = bempp.api.GridFunction(space=space_unit_sphere, fun=far_fun)
far_coeff = far_grid.coefficients

In [12]:
deg = 5

def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

sph_harm_coeff = []
for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere, fun = sph_harm)
        sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)* k * ((1j)**(q+1)) )

In [21]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_unit_sphere, space_unit_sphere, space_unit_sphere).weak_form().A

test_list = []
for i in range((deg+1)**2):
    test_list.append(far_coeff @ mass_mat @ sph_harm_coeff[i])

In [22]:
def sca_far_field_test(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += test_list[i]*(1/k)*(-1j)**(q+1) * normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

In [23]:
sca_far_field_test(vert_unit_sphere[:,0])

(-1.347362922587048+2.1948262534192327j)

#### Numerical experiment on finding the scattered field and far field

Method 1

In [9]:
eta = 0.5

identity = bempp.api.operators.boundary.sparse.identity(space_sphere, space_sphere, space_sphere)
slp = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere, space_sphere, space_sphere,k)
adlp = bempp.api.operators.boundary.helmholtz.adjoint_double_layer(space_sphere, space_sphere, space_sphere,k)

slp_pot = bempp.api.operators.potential.helmholtz.single_layer(space_sphere, vert_big_sphere, k)
slp_far = bempp.api.operators.far_field.helmholtz.single_layer(space_sphere, vert_unit_sphere, k)

lhs = 0.5 * identity + adlp - 1j *eta* slp

In [10]:
@bempp.api.complex_callable
def combined_data(x, n, domain_index, result):
    result[0] = 1j * k * np.exp(1j * k * x[0]) * n[0] - 1j * eta * np.exp(1j * k * x[0])

grid_fun = bempp.api.GridFunction(space_sphere, fun=combined_data)
neumann_fun, info = bempp.api.linalg.gmres(lhs, grid_fun, tol=1E-5)

res_near = - slp_pot.evaluate(neumann_fun)
res_far = -slp_far.evaluate(neumann_fun)

print(res_near[0,0], res_far[0,0])

(-0.7688451025888488-0.3051535820643964j) (-0.9297504949612159+0.30912696727427347j)


In [11]:
np.linalg.norm(res_far[0,0] - far_field(vert_unit_sphere[:,0]))/np.linalg.norm(far_field(vert_unit_sphere[:,0]))

0.005261836889734651

Method 2 

In [117]:
eta2 = k

dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_sphere,space_sphere,space_sphere,k)

dlp_pot = bempp.api.operators.potential.helmholtz.double_layer(space_sphere, vert_big_sphere , k)
dlp_far =  bempp.api.operators.far_field.helmholtz.double_layer(space_sphere, vert_unit_sphere,k)

lhs2 = 0.5 * identity + dlp - 1j* eta2 *slp

In [118]:
@bempp.api.complex_callable
def data(x, n, domain_index, result):
    result[0] = - np.exp(1j * k * x[0])

grid_fun2 = bempp.api.GridFunction(space_sphere, fun=data)
fun2, info2 = bempp.api.linalg.gmres(lhs2, grid_fun2, tol=1E-5)

res_near2 = dlp_pot.evaluate(fun2) - 1j * eta2 * slp_pot.evaluate(fun2)
res_far2 = dlp_far.evaluate(fun2) - 1j * eta2 * slp_far.evaluate(fun2)

print(res_near2[0,0], res_far2[0,0])

(0.5414497471031161-0.8307398397007305j) (-9.817315335167095+45.125159140083056j)


### Compute T matrix and Check

In [78]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [79]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_unit_sphere,space_unit_sphere,space_unit_sphere).weak_form().A

In [102]:
deg = 10
coeff_scattered_field = []
for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere,fun = sph_harm)
        coeff_scattered_field.append(k* (1j**(q+1)) * (res_far2 @ mass_mat @ np.conj(sph_harm_grid.coefficients)))

In [81]:
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*(1j)**q*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))
    return coeff

In [82]:
coeff_inc = []
for q in range(deg+1):
    for p in range(-q, q+1):
        coeff_inc.append(coeff_sph_expansion(p,q))

In [103]:
slp_far_field_coeff = []
sph_harm_coeff = []

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def dirichlet_fun(x,n,domain_index,result):
                with objmode():
                    result[0] = - regular_spherical_wavefunctions(p,q,x)

        rhs_fun = bempp.api.GridFunction(space_sphere, fun = dirichlet_fun)
        field, info = bempp.api.linalg.gmres(lhs2, rhs_fun, tol=1E-5)
        slp_far_field_coeff.append(dlp_far.evaluate(field) - 1j * eta * slp_far.evaluate(field))

        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere,fun = sph_harm)
        sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)/((-1j)**(q+1)))

In [104]:
T_matrix = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
for i in range((deg+1)**2):
    for j in range((deg+1)**2):
        T_matrix[i,j] = (slp_far_field_coeff[j] @ mass_mat @ sph_harm_coeff[i])*k

In [108]:
def sca_far_field(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += coeff_scattered_field[i]*(1/k)*(-1j)**(q+1) * normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

In [106]:
coeff_sca = T_matrix@ coeff_inc

In [109]:
sca_far_field(vert_unit_sphere[:,0])

array([-0.90900834+12.12613233j])

In [ ]:
T_matrix + np.matrix(T_matrix).getH() + 2* T_matrix @ np.matrix(T_matrix).getH()

In [110]:
T_matrix[3,3]

(-0.1332425493384566+0.3375824005658579j)

In [111]:
T_matrix[1,1]

(-0.13324159798059848+0.3375823105443226j)